# Compare time series from a datacube

Import PyOphidia and connect to server instance

In [ ]:
from PyOphidia import cube, client
cube.Cube.setclient(read_env=True)

Import data, compute difference between a time series from 2nd year and one from 1st year, and get output

In [ ]:
mycube = cube.Cube.importnc(src_path='/data/tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',measure='tasmax',imp_dim='time',ncores=4)
firstYear = mycube.subset(subset_dims="lat|lon|time", subset_type="coord", subset_filter="1|1|0:366",ncores=4)
secondYear = mycube.subset(subset_dims="lat|lon|time", subset_type="coord", subset_filter="1|1|366:731",ncores=4)

firstYear_avg = firstYear.reduce2(
    operation='avg',
    dim='time',
    concept_level='M',
)
secondYear_avg = secondYear.reduce2(
    operation='avg',
    dim='time',
    concept_level='M',
)
diff = secondYear_avg.intercube(cube2=firstYear_avg.pid)
data = diff.export_array()

Plot time series difference

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
y = data['measure'][0]['values'][0][:]
x = data['dimension'][2]['values'][:]
plt.figure(figsize=(11, 5), dpi=100)
plt.grid(zorder=0)
plt.bar(x, y, width=10, zorder=2)

plt.ylabel(data['measure'][0]['name'] + " (degK)")
plt.title('Maximum Near-Surface Air Temperature - difference 2096-2097')
plt.xticks(x, ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], rotation='vertical')
plt.show()

Clear your workspace before running other notebooks

In [ ]:
cube.Cube.deletecontainer(container='tasmax_day_CMCC-CESM_rcp85_r1i1p1_20960101-21001231.nc',force='yes')